In [1]:
!pip install transformers --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install langchain --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.
cudf 24.6.1 requires cuda-python<12.0a0,>=11.7.1, but you have cu

In [2]:
pip install langchain-community langchain-core --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
# General packages
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from textwrap import fill
from IPython.display import Markdown, display # for formating Python display folowing markdown language
import warnings
warnings.filterwarnings('ignore') # avoid warning messages importing packages

In [5]:
# Mistral and LangChain packages (prompt engineering)
import torch
from langchain import PromptTemplate, HuggingFacePipeline
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

In [7]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGING_FACE_WRITE")
secret_value_1 = user_secrets.get_secret("HUGGINGFACE_TOKEN")

In [9]:
!huggingface-cli login --token $secret_value_0

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
# Model version of Mistral
MODEL_NAME = "AbidaRashid/mistral_7b_mlec"

# Quantization is a technique used to reduce the memory and computation requirements 
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the Mistral-7b model, 
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language Mistral-7b
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 20 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.7 # randomness of the generated tex
generation_config.top_p = 0.95 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [11]:
# HuggingFace pipeline
llm = HuggingFacePipeline(pipeline=pipe)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [12]:
import re
# Define the emotion labels
subject_labels =  ['computer science','physics', 'mathematics', 'statistics', 'quantitative biology', 'quantitative finance']

In [13]:
def extract_subject(text):
    """Extracts text outside [INST] and [/INST] tags."""
    last_inst_index = text.rfind("[/INST]")
    if last_inst_index != -1:
        my_text = text[last_inst_index + len("[/INST]") :]
        my_text= my_text.lower()
        #print(my_text, "%%%%%%%%")
        response_text = my_text.replace('[', ' ').replace('\n',' ').replace(']', ' ').replace('\'', ' ').replace('.',' ').replace('\"',' ').replace(',',' ')
        detected_subject = response_text.split(" ")
        #print('***********',detected_emotions)
        return [1 if subject in detected_subject else 0 for subject in subject_labels]
    else:
        return [0] * len(subject_labels)



In [14]:
#/kaggle/input/mlec-english-sample/Train_Data_Sample.csv

import pandas as pd
import csv
data= pd.read_csv('/kaggle/input/abstract-multi-label-dataset/Test_Sample.csv')
data_t = data.copy()
#Extract tweet text from dataset
test_texts = data_t['ABSTRACT'].tolist()

#Extract actual_values for emotions
test_labels = data_t.drop(columns=['ABSTRACT','TITLE','ID']).values

In [15]:
predictions = []
for text in test_texts:
    #text = "i want to go out  of this house"
    # In English
    query = f"""[INST]Being an expert who can detect subject in a given abstract
        enclosed in “” please read the provided abstract. 
        Identify and list the subject that the provided abstract might be of or closest to, list only one
        Consider the following subjects: 
        Computer Science
        Physics
        Mathematics
        Statistics
        Quantitative Biology
        Quantitative Finance
        Other
        If no subjects are detected or the abstract is unclear, return ['Other']. 
        your answer should be in the form of a Python list containing subject label only mentioned above and presented like as an example
        given below:
        Subject : ['Statistics']
            Text: {text}
            Subject: 
            .[/INST] """
    result = llm(query)
    #display(Markdown(f"<b>{query.removeprefix('[INST]').removesuffix('[/INST]')}</b>"))
    display(Markdown(f"<p>{result}</p>"))
    r_label = extract_subject(result)
    print("****", r_label)
    predictions.append(r_label)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


<p>[INST]Being an expert who can detect subject in a given abstract
        enclosed in “” please read the provided abstract. 
        Identify and list the subject that the provided abstract might be of or closest to, list only one
        Consider the following subjects: 
        Computer Science
        Physics
        Mathematics
        Statistics
        Quantitative Biology
        Quantitative Finance
        Other
        If no subjects are detected or the abstract is unclear, return ['Other']. 
        your answer should be in the form of a Python list containing subject label only mentioned above and presented like as an example
        given below:
        Subject : ['Statistics']
            Text:   We consider the problem of estimating the $L_1$ distance between two discrete
probability measures $P$ and $Q$ from empirical data in a nonasymptotic and
large alphabet setting. When $Q$ is known and one obtains $n$ samples from $P$,
we show that for every $Q$, the minimax rate-optimal estimator with $n$ samples
achieves performance comparable to that of the maximum likelihood estimator
(MLE) with $n\ln n$ samples. When both $P$ and $Q$ are unknown, we construct
minimax rate-optimal estimators whose worst case performance is essentially
that of the known $Q$ case with $Q$ being uniform, implying that $Q$ being
uniform is essentially the most difficult case. The \emph{effective sample size
enlargement} phenomenon, identified in Jiao \emph{et al.} (2015), holds both in
the known $Q$ case for every $Q$ and the $Q$ unknown case. However, the
construction of optimal estimators for $\|P-Q\|_1$ requires new techniques and
insights beyond the approximation-based method of functional estimation in Jiao
\emph{et al.} (2015).

            Subject: 
            .[/INST]  ["Physics"]  ["Quantitative Biology"]  ["Other"]  ["Computer Science</p>

**** [0, 1, 0, 0, 0, 0]


<p>[INST]Being an expert who can detect subject in a given abstract
        enclosed in “” please read the provided abstract. 
        Identify and list the subject that the provided abstract might be of or closest to, list only one
        Consider the following subjects: 
        Computer Science
        Physics
        Mathematics
        Statistics
        Quantitative Biology
        Quantitative Finance
        Other
        If no subjects are detected or the abstract is unclear, return ['Other']. 
        your answer should be in the form of a Python list containing subject label only mentioned above and presented like as an example
        given below:
        Subject : ['Statistics']
            Text:   We investigate the density large deviation function for a multidimensional
conservation law in the vanishing viscosity limit, when the probability
concentrates on weak solutions of a hyperbolic conservation law conservation
law. When the conductivity and dif-fusivity matrices are proportional, i.e. an
Einstein-like relation is satisfied, the problem has been solved in [4]. When
this proportionality does not hold, we compute explicitly the large deviation
function for a step-like density profile, and we show that the associated
optimal current has a non trivial structure. We also derive a lower bound for
the large deviation function, valid for a general weak solution, and leave the
general large deviation function upper bound as a conjecture.

            Subject: 
            .[/INST]  ["Physics"]  ["Quantitative Biology"]  ["Computer Science"]  ["Mat</p>

**** [0, 1, 0, 0, 0, 0]


...

**** [0, 1, 0, 0, 0, 0]


<p>[INST]Being an expert who can detect subject in a given abstract
        enclosed in “” please read the provided abstract. 
        Identify and list the subject that the provided abstract might be of or closest to, list only one
        Consider the following subjects: 
        Computer Science
        Physics
        Mathematics
        Statistics
        Quantitative Biology
        Quantitative Finance
        Other
        If no subjects are detected or the abstract is unclear, return ['Other']. 
        your answer should be in the form of a Python list containing subject label only mentioned above and presented like as an example
        given below:
        Subject : ['Statistics']
            Text:   We study the effect of domain growth on the orientation of striped phases in
a Swift-Hohenberg equation. Domain growth is encoded in a step-like parameter
dependence that allows stripe formation in a half plane, and suppresses
patterns in the complement, while the boundary of the pattern-forming region is
propagating with fixed normal velocity. We construct front solutions that leave
behind stripes in the pattern-forming region that are parallel to or at a small
oblique angle to the boundary.
Technically, the construction of stripe formation parallel to the boundary
relies on ill-posed, infinite-dimensional spatial dynamics. Stripes forming at
a small oblique angle are constructed using a functional-analytic, perturbative
approach. Here, the main difficulties are the presence of continuous spectrum
and the fact that small oblique angles appear as a singular perturbation in a
traveling-wave problem. We resolve the former difficulty using a farfield-core
decomposition and Fredholm theory in weighted spaces. The singular perturbation
problem is resolved using preconditioners and boot-strapping.

            Subject: 
            .[/INST]  ["Mathematics"]  ["Physics"]  ["Computer Science"]  ["Quantitative</p>

**** [0, 1, 1, 0, 0, 0]


<p>[INST]Being an expert who can detect subject in a given abstract
        enclosed in “” please read the provided abstract. 
        Identify and list the subject that the provided abstract might be of or closest to, list only one
        Consider the following subjects: 
        Computer Science
        Physics
        Mathematics
        Statistics
        Quantitative Biology
        Quantitative Finance
        Other
        If no subjects are detected or the abstract is unclear, return ['Other']. 
        your answer should be in the form of a Python list containing subject label only mentioned above and presented like as an example
        given below:
        Subject : ['Statistics']
            Text:   This paper discusses minimum distance estimation method in the linear
regression model with dependent errors which are strongly mixing. The
regression parameters are estimated through the minimum distance estimation
method, and asymptotic distributional properties of the estimators are
discussed. A simulation study compares the performance of the minimum distance
estimator with other well celebrated estimator. This simulation study shows the
superiority of the minimum distance estimator over another estimator. KoulMde
(R package) which was used for the simulation study is available online. See
section 4 for the detail.

            Subject: 
            .[/INST] 
            ["Mathematics"] 
            ["Physics"] 
            ["Quantitative</p>

**** [0, 1, 1, 0, 0, 0]


In [16]:
for i in range(20):
    print(test_labels[i],"\n",predictions[i], "\n\n")
#print(test_labels, "\n\n", predictions)

[0 0 1 1 0 0] 
 [0, 1, 0, 0, 0, 0] 


[0 1 1 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 0 1 0 0] 
 [0, 0, 0, 0, 0, 0] 


[0 0 1 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [0, 0, 1, 0, 0, 0] 


[1 0 1 0 0 0] 
 [0, 0, 0, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 1 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [0, 0, 1, 0, 0, 0] 


[1 0 0 1 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 0 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[0 0 1 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 0 1 0 0] 
 [0, 1, 0, 1, 0, 0] 


[0 0 1 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[0 0 1 1 0 0] 
 [0, 1, 1, 0, 0, 0] 


[0 1 0 0 0 0] 
 [0, 1, 0, 0, 0, 0] 


[1 0 1 1 0 0] 
 [0, 1, 0, 0, 0, 0] 




In [17]:
from sklearn.metrics import f1_score, precision_score, recall_score
# Compute evaluation metrics
f1_micro = f1_score(test_labels, predictions, average='micro')
f1_macro = f1_score(test_labels, predictions, average='macro')
precision_micro = precision_score(test_labels, predictions, average='micro')
precision_macro = precision_score(test_labels, predictions, average='macro')
recall_micro = recall_score(test_labels, predictions, average='micro')
recall_macro = recall_score(test_labels, predictions, average='macro')

# Print the evaluation metrics
print(f"F1 Micro: {f1_micro}")
print(f"F1 Macro: {f1_macro}")
print(f"Precision Micro: {precision_micro}")
print(f"Precision Macro: {precision_macro}")
print(f"Recall Micro: {recall_micro}")
print(f"Recall Macro: {recall_macro}")

F1 Micro: 0.3333333333333333
F1 Macro: 0.17585470085470087
Precision Micro: 0.4
Precision Macro: 0.29473684210526313
Recall Micro: 0.2857142857142857
Recall Macro: 0.2238095238095238
